In [1]:
%%capture
import sys
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a
sys.path.append("../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT")
sys.path.append("../input/nvidiaapex/repository/NVIDIA-apex-39e153a")

In [2]:
import numpy as np
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from torch.utils import data
#from tqdm import tqdm_notebook as tqdm

import random
import os

import torch
import torch.utils.data
import torch.nn as nn
from apex import amp
from pytorch_pretrained_bert import BertTokenizer, BertAdam, BertConfig, convert_tf_checkpoint_to_pytorch, BertForSequenceClassification

In [3]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
MAX_SEQUENCE_LENGTH = 220

Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
TOXICITY_COLUMN = 'target'
output_model_file = "bert_pytorch.bin"

EPOCHS = 1
checkpoint_weights = [1, 2, 4][:EPOCHS]
lr=2e-5
batch_size = 8
accumulation_steps=4

In [4]:
bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')
device=torch.device('cuda')
tqdm.pandas()

In [5]:
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [6]:
#train['binned_target'] = pd.cut(train['target'], [0, 0.25, 0.5, 0.75, 1.1], right=False, labels=[1, 2, 3, 4])

In [7]:
# train1 = train[train['binned_target'] == 1]#.sample(n=38368, random_state=123)
# train2 = train[train['binned_target'] == 2]#.sample(n=105966, random_state=123)
# train3 = train[train['binned_target'] == 3]
# train4 = train[train['binned_target'] == 4]

# train1 = train1.sample(n=len(train1)//2, random_state=1234)
# train2 = train2.sample(n=len(train2)//2, random_state=1234)
# train3 = train3.sample(n=len(train3), random_state=1234)
# train4 = train4.sample(n=len(train4), random_state=1234)

# #train = pd.concat([train1, train2, train3, train4])

# train = train.sample(n=len(train)//2, random_state=123)

In [8]:
%%capture
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

In [9]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [10]:
def preprocess(x, token_length, tokenizer):
    tokens_a = tokenizer.tokenize(x)[:token_length]
    one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (token_length - len(tokens_a))
    return np.array(one_token)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

x_train = train['comment_text'].progress_apply(lambda x:preprocess(x, MAX_SEQUENCE_LENGTH-2, tokenizer))
x_test = test['comment_text'].progress_apply(lambda x:preprocess(x, MAX_SEQUENCE_LENGTH-2, tokenizer))

x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)

identity_columns = ['severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
y_aux_train = train[identity_columns]

In [ ]:
# x_train = tokenizer.texts_to_sequences(x_train)
# x_test = tokenizer.texts_to_sequences(x_test)
# lengths = torch.from_numpy(np.array([len(x) for x in x_train]))
# maxlen = 300 
# x_train_padded = torch.from_numpy(sequence.pad_sequences(x_train, maxlen=maxlen))
# x_train_padded.shape

In [ ]:
# test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))
# x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=maxlen))

In [ ]:
# class SequenceBucketCollator():
#     def __init__(self, choose_length, sequence_index, length_index, label_index=None):
#         self.choose_length = choose_length
#         self.sequence_index = sequence_index
#         self.length_index = length_index
#         self.label_index = label_index
        
#     def __call__(self, batch):
#         batch = [torch.stack(x) for x in list(zip(*batch))]
        
#         sequences = batch[self.sequence_index]
#         lengths = batch[self.length_index]
        
#         length = self.choose_length(lengths)
#         mask = torch.arange(start=maxlen, end=0, step=-1) < length
#         padded_sequences = sequences[:, mask]
        
#         batch[self.sequence_index] = padded_sequences #BERT tokenize this
        
#         if self.label_index is not None:
#             return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
#         return batch

In [ ]:
# Overall
weights = np.ones((len(x_train),)) / 4

# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4

# Background Positive, Subgroup Negative
weights += (((train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [ ]:
loss_weight = 1.0 / weights.mean()
y_train = np.vstack([(train['target'].values>=0.5), weights]).T

In [ ]:
model = BertForSequenceClassification(bert_config,num_labels=len(identity_columns) + 1)
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model = model.train()

In [ ]:
class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):

        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an inccorect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
    max_len = torch.max(torch.sum( (tsrs[0] != 0  ), 1))
    if max_len > 2: 
        tsrs = [tsr[:, :max_len] for tsr in tsrs]
    return tsrs 

In [ ]:
tq = tqdm(range(EPOCHS))
training_dataset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.long), torch.tensor(np.hstack([y_train, y_aux_train]),dtype=torch.float))
testing_dataset = torch.utils.data.TensorDataset(torch.tensor(x_test, dtype=torch.long))
#train_collator = SequenceBucketCollator(lambda lengths: lengths.max(), sequence_index=0, length_index=1, label_index=2)
#test_collator = SequenceBucketCollator(lambda lengths: lengths.max(), sequence_index=0, length_index=1)

test_loader = train_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=batch_size, shuffle=False)

all_test_preds = []


for epoch in tq:
    #train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    
    ran_sampler = data.RandomSampler(training_dataset)
    len_sampler = LenMatchBatchSampler(ran_sampler, batch_size = 16, drop_last = False)
    train_loader = torch.utils.data.DataLoader(training_dataset, batch_sampler = len_sampler) 
    
    tk0 = tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
    for i, (x_batch, y_batch) in tk0:
        optimizer.zero_grad()
        
        #tsrs = trim_tensors(batch)
        #x_batch, b_input_mask, b_segment_ids, y_batch = tuple(t.to(device) for t in tsrs)
        
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  custom_loss(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    
    test_preds = np.zeros((len(test), len(['target'] + identity_columns)))
    tk1 = tqdm(enumerate(test_loader),total=len(test_loader),leave=False)
    for i, x_batch in tk1:
        X = x_batch[0].cuda()
        y_pred = model(torch.tensor(X)).detach().cpu().numpy()
        test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred
        
    all_test_preds.append(test_preds)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

#final_pred = np.average(all_test_preds, weights=checkpoint_weights, axis=0)  
final_pred = all_test_preds[0]

torch.save(model.state_dict(), output_model_file)

In [ ]:
submission = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': final_pred[:, 0],
})
submission.to_csv('submission.csv', index=False)